In [7]:
import json
import twitter
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream, auth
import pandas as pd
import re, collections
import time
CONSUMER_KEY = 'JJNPMPuZubIrFIqDNARJRDPDb'
CONSUMER_SECRET = 'd6MlAqrJFcdyJac9aCQpBHPbwGs5eJB8zkTn5wA3tqBLHZgw0b'
OAUTH_TOKEN = '3229899732-piSMFy32Vi0VSyXJX8R9y2qrkr0piesoHXBdI3v'
OAUTH_TOKEN_SECRET = 'Jq9oTRMUjHRgA7NkJLLHIEyjtCRhYiFHdWkpBw28IBtHG'

#read emoji codes:
emoji_key = pd.read_excel('/Users/attiladobi/Documents/emoji_list.xlsx', encoding='utf-8', index_col=0, skiprows=1)
emj_codes=[code for code in emoji_key['Unicode'] if code!="Browser" if sum([c=="*" for c in code])==0]

def get_keys(tweet):
    max_index=0
    max_count=len(tweet['statuses'][max_index].keys())
    
    for ii in range(len(tweet['statuses'])):
        if max_count < len(tweet['statuses'][ii].keys()):
            max_index=ii
            
    return tweet['statuses'][max_index].keys()

def print_tweets_w_emj(T_DF,num=0):
    for item in T_DF['text'][T_DF['emjSum']>=num]:
        print(item)
        
def make_twitter_DF(tweet): #create PD dataframe
    columns=get_keys(tweet)
    for ii in range(len(tweet['statuses'])): 
        diff= list(set(columns)-set(tweet['statuses'][ii].keys()))
        for val in diff:
            tweet['statuses'][ii][val]=""
    return pd.DataFrame([tweet['statuses'][ii] for ii in range(len(tweet['statuses']))], columns=get_keys(tweet)) 

def Search_Word(q):
    # Initiate the connection to Twitter REST API
    twitter = Twitter(auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))       
    tweet= twitter.search.tweets(q=q,lang='en', result_type='recent',count=200)
    T_DF=make_twitter_DF(tweet)
    
    #long search function, more than 100
    #T_DF=long_search(twitter,q,300)
    
    T_DF['emjText']=[[(emcode, len(re.findall(emcode,T_DF['text'][ii]))) for emcode in emj_codes\
                      if (len(re.findall(emcode,T_DF['text'][ii])) > 0)] for ii in range(len(T_DF))]
    T_DF['emjSum']=[sum([item[1] for item in T_DF['emjText'][ii]]) for ii in range(len(T_DF))]
    T_DF['emjTypes']=[len(T_DF['emjText'][ii]) for ii in range(len(T_DF))]
    T_DF['searchWordSum']=[len(re.findall(q,T_DF['text'][ii].lower())) for ii in range(len(T_DF))]
    return T_DF

def Analyze_Freq(T_DF,emj_per_tweet=1,rep_emj_per_tweet=1, emj_types_per_tweet=1, nword=0):
    DFcut=T_DF[(T_DF['emjSum']>=emj_per_tweet) & (T_DF['searchWordSum']>=nword) \
               & (T_DF['emjTypes']>=emj_types_per_tweet)] 
    
    S=dict()
    for key, value in sum(DFcut['emjText'],[]):
        if value>=rep_emj_per_tweet:
            try:
                S[key] +=value
            except KeyError:
                S[key]=value
        
    return pd.DataFrame([(key,S[key]) for key in S],columns=('emoji','freq'))

def count_words(text):
    S = collections.defaultdict(lambda:0)
    for word in text.split():
        S[word.lower()]+=1
    return max(S, key=S.get), S[max(S, key=S.get)]

<h1> Code to stream twitter and filter on emoji </h1>

In [17]:
def tweet_stream():
    twitter_stream = TwitterStream(domain='stream.twitter.com', auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))
    iterator = twitter_stream.statuses.sample() #track='dance', language="en"
    return iterator

In [9]:
columns = ['created_at', 'text', 'retweet_count', 'favorite_count', 'lang','geo', 'coordinates', \
           'emjText', 'emjCount','emjTypes','mostFreqEmoji','mostFreqEmojiCount','mostFreqWord','mostFreqWordCount']
df= pd.DataFrame(columns=columns)

In [10]:
#if ('hangup' in tweet.keys()): ... connect again

def mine_for_emojis(iterator=tweet_stream()):
    tweet_count = 100
    for tweet in iterator:
        if ('text' in tweet.keys()):

            emjText=[(emcode, len(re.findall(emcode,tweet['text']))) for emcode in emj_codes\
                              if (len(re.findall(emcode,tweet['text'])) > 0)]

            if len(emjText) >0:
                tweet_count -= 1

                emjCount=sum([item[1] for item in emjText])
                emjTypes = len(emjText)
                a=np.array(emjText)
                mostFreqEmoji = a[np.argsort(a[:, 1])][-1][0]
                mostFreqEmojiCount = int(a[np.argsort(a[:, 1])][-1][1])
                mostFreqWord, mostFreqWordCount = count_words(tweet['text'])

                df.loc[len(df)] = [tweet['created_at'], tweet['text'], tweet['retweet_count'], tweet['favorite_count'], \
                                   tweet['lang'],tweet['geo'], tweet['coordinates'], \
                                   emjText, emjCount,emjTypes,mostFreqEmoji, mostFreqEmojiCount, mostFreqWord, mostFreqWordCount]

                print(tweet['text'])

            if tweet_count <= 0:
                break
        else:
            print('hungup')
            time.sleep(30)
            mine_for_emojis()


In [37]:
good_tweets=[]
twitter_stream = TwitterStream(domain='stream.twitter.com', auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))
iterator = twitter_stream.statuses.sample() #track='dance', language="en"
for tweet in iterator:
    if (tweet['user']['geo_enabled']):
        print tweet
        good_tweets.append(tweet)

{u'favorited': False, u'contributors': None, u'truncated': False, u'text': u'RT @elbidondebranco: Jugando asi San Lorenzo no solo no merece pasar de ronda, merece descender.', u'is_quote_status': False, u'in_reply_to_status_id': None, u'user': {u'follow_request_sent': None, u'profile_use_background_image': True, u'default_profile_image': False, u'id': 2774331967, u'verified': False, u'profile_image_url_https': u'https://pbs.twimg.com/profile_images/686960308456390656/WHHfkEVd_normal.jpg', u'profile_sidebar_fill_color': u'000000', u'profile_text_color': u'000000', u'followers_count': 3437, u'profile_sidebar_border_color': u'000000', u'id_str': u'2774331967', u'profile_background_color': u'000000', u'listed_count': 10, u'profile_background_image_url_https': u'https://pbs.twimg.com/profile_background_images/618530923831648257/Q5LUlydG.jpg', u'utc_offset': -10800, u'statuses_count': 15104, u'description': u'Fuerza de choque del movimiento Fantinista Argentino Revolucionario.\nDel humo veni

KeyError: 'user'

In [38]:
tweet['user']['screen_name']
tweet['user']['geo_enabled']

KeyError: 'user'

In [42]:
good_tweets[0]['text']

u'RT @elbidondebranco: Jugando asi San Lorenzo no solo no merece pasar de ronda, merece descender.'

In [12]:
mine_for_emojis()

hungup


KeyboardInterrupt: 

In [43]:
#Good cut for unique emojis

df['text'][(df['emjTypes']==df['emjCount']) & (df['emjCount']>=3) ]

Series([], Name: text, dtype: object)

<h1> Write to mongo DB </h1>

In [294]:
import datetime
from pymongo import MongoClient

client = MongoClient()
db = client.emoji_db
collection = db.emoji_tweets


In [295]:
entry = {"date": datetime.datetime.utcnow(),\
         "created_at": tweet['created_at'],\
         "text": tweet['text'],\
         "retweet_count": tweet['retweet_count'],\
         "favorite_count": tweet['favorite_count'], \
         "lang": tweet['lang'],\
         "goe": tweet['geo'],\
         "coordinates": tweet['coordinates'], \
         "emjText": emjText, "emjCount": emjCount, "emjTypes": emjTypes, "mostFreqEmoji": mostFreqEmoji,\
         "mostFreqEmojiCount": mostFreqEmojiCount, "mostFreqWord": mostFreqWord,\
         "mostFreqWordCount": mostFreqWordCount}

In [296]:
collection.insert_one(entry).inserted_id

ObjectId('56edad8dc2182c74ba76ce17')

<h1> Tweepy </h1>

In [575]:
from __future__ import division
import json
import tweepy
from tweepy.streaming import StreamListener
#from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream, auth
import pandas as pd
import re, collections
import numpy as np
CONSUMER_KEY = 'JJNPMPuZubIrFIqDNARJRDPDb'
CONSUMER_SECRET = 'd6MlAqrJFcdyJac9aCQpBHPbwGs5eJB8zkTn5wA3tqBLHZgw0b'
OAUTH_TOKEN = '3229899732-piSMFy32Vi0VSyXJX8R9y2qrkr0piesoHXBdI3v'
OAUTH_TOKEN_SECRET = 'Jq9oTRMUjHRgA7NkJLLHIEyjtCRhYiFHdWkpBw28IBtHG'

import os
base_dir=os.path.expanduser('~')

import datetime
import time
from pymongo import MongoClient
#setup mongo DB
client = MongoClient()
db = client.emoji_db
collection = db.emoji_tweets
emoji_usage = db.emoji_usage

#read emoji codes:
emoji_key = pd.read_excel(base_dir+'/emojify/data/emoji_list.xlsx', encoding='utf-8', index_col=0, skiprows=1)
emj_codes_skin=[code for code,name in zip(emoji_key['Unicode'],emoji_key['Name']) if ('FITZPATRICK' in name)]
emj_codes=[code for code in emoji_key['Unicode'] if code!="Browser" \
           if (code not in emj_codes_skin) if sum([c=="*" for c in code])==0]
#remove common face emojis
noise_index=range(69)
emj_codes_noise=[code for index,code in zip(emoji_key.index,emoji_key['Unicode']) if index in noise_index]

def get_keys(tweet):
    max_index=0
    max_count=len(tweet['statuses'][max_index].keys())
    
    for ii in range(len(tweet['statuses'])):
        if max_count < len(tweet['statuses'][ii].keys()):
            max_index=ii
            
    return tweet['statuses'][max_index].keys()

def print_tweets_w_emj(T_DF,num=0):
    for item in T_DF['text'][T_DF['emjSum']>=num]:
        print(item)
        
def make_twitter_DF(tweet): #create PD dataframe
    columns=get_keys(tweet)
    for ii in range(len(tweet['statuses'])): 
        diff= list(set(columns)-set(tweet['statuses'][ii].keys()))
        for val in diff:
            tweet['statuses'][ii][val]=""
    return pd.DataFrame([tweet['statuses'][ii] for ii in range(len(tweet['statuses']))], columns=get_keys(tweet)) 

def Search_Word(q):
    # Initiate the connection to Twitter REST API
    twitter = Twitter(auth=OAuth( OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY,  CONSUMER_SECRET))       
    tweet= twitter.search.tweets(q=q,lang='en', result_type='recent',count=200)
    T_DF=make_twitter_DF(tweet)
    
    #long search function, more than 100
    #T_DF=long_search(twitter,q,300)
    
    T_DF['emjText']=[[(emcode, len(re.findall(emcode,T_DF['text'][ii]))) for emcode in emj_codes\
                      if (len(re.findall(emcode,T_DF['text'][ii])) > 0)] for ii in range(len(T_DF))]
    T_DF['emjSum']=[sum([item[1] for item in T_DF['emjText'][ii]]) for ii in range(len(T_DF))]
    T_DF['emjTypes']=[len(T_DF['emjText'][ii]) for ii in range(len(T_DF))]
    T_DF['searchWordSum']=[len(re.findall(q,T_DF['text'][ii].lower())) for ii in range(len(T_DF))]
    return T_DF

def Analyze_Freq(T_DF,emj_per_tweet=1,rep_emj_per_tweet=1, emj_types_per_tweet=1, nword=0):
    DFcut=T_DF[(T_DF['emjSum']>=emj_per_tweet) & (T_DF['searchWordSum']>=nword) \
               & (T_DF['emjTypes']>=emj_types_per_tweet)] 
    
    S=dict()
    for key, value in sum(DFcut['emjText'],[]):
        if value>=rep_emj_per_tweet:
            try:
                S[key] +=value
            except KeyError:
                S[key]=value
        
    return pd.DataFrame([(key,S[key]) for key in S],columns=('emoji','freq'))

def count_words(text):
    S = collections.defaultdict(lambda:0)
    for word in text.split():
        S[word.lower()]+=1
    return max(S, key=S.get), S[max(S, key=S.get)]

def emoji_split(text):
    '''add a space after the emoji, and remove double spaces'''
    for emcode in emj_codes:
        if (len(re.findall(emcode,text))) > 0:
            text=text.replace(emcode,emcode+' ')
    for skin in emj_codes_skin:
        text=text.replace(' '+skin,skin+' ') #put the skin codes back into place but add a space afterwards
    return ' '.join(text.split())

def NextWord(text,emj):
    try:
        last_index=len(text)-(text[::-1].index(emj[::-1]))-2
    except ValueError:
        last_index=len(text)
    return ''.join(text[last_index+2:].split()[:1])

    
class StdOutListener(StreamListener):
    """ A listener handles tweets that are received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    def on_status(self, status):
        mine_for_emojis(status)
        return True

    def on_error(self, status):
        print(status)

def write_emoji_usage(tweet,has_emoji):
    entry = {"date": datetime.datetime.utcnow(),\
             "created_at": tweet.created_at,\
             "retweet_count": tweet.retweet_count,\
             "favorite_count": tweet.favorite_count,\
             "lang": tweet.lang,\
             "goe": tweet.geo,\
             "coordinates": tweet.coordinates,\
             "time_zone": tweet.user.time_zone,\
             "name":tweet.user.name, "user_name":tweet.user.screen_name,\
             "has_emoji":has_emoji}
    emoji_usage.insert_one(entry).inserted_id
 
        
def mine_for_emojis(tweet):
    
    has_emoji=False
    emjText=[(emcode, len(re.findall(emcode,tweet.text))) for emcode in emj_codes\
                      if (len(re.findall(emcode,tweet.text)) > 0)]
    
    if len(emjText) >0:
        has_emoji=True
        emjCount=sum([item[1] for item in emjText])
        emjTypes = len(emjText)
        a=np.array(emjText)
        mostFreqEmoji = a[np.argsort(a[:, 1])][-1][0]
        mostFreqEmojiCount = int(a[np.argsort(a[:, 1])][-1][1])
        mostFreqWord, mostFreqWordCount = count_words(tweet.text)
        newlineCount= tweet.text.count('\n')
        surrounding_text = [(emcode, tweet.text[:tweet.text.index(emcode)].split()[-1:],\
                tweet.text[tweet.text.index(emcode)+2:].split()[:1],\
                tweet.text[:tweet.text.index(emcode)].split()[-5:],\
                tweet.text[tweet.text.index(emcode)+2:].split()[:5]) for emcode in emj_codes\
                  if (len(re.findall(emcode,tweet.text)) > 0)]
        
        prev_word = ''.join(tweet.text[:tweet.text.index(mostFreqEmoji)].split()[-1:])
        #finding the next word is more tricky :)
        try:
            last_index=len(tweet.text)-(tweet.text[::-1].index(mostFreqEmoji[::-1]))-2
        except ValueError:
            last_index=len(tweet.text)
        next_word = ''.join(tweet.text[last_index+2:].split()[:1])
        
        #skin tone information
        emjText_skin=[(emcode, len(re.findall(emcode,tweet.text))) for emcode in emj_codes_skin\
                      if (len(re.findall(emcode,tweet.text)) > 0)]
        skinCount=sum([item[1] for item in emjText_skin])
        skinTypes = len(emjText_skin)
        if skinTypes==0:
            mostFreqSkin, mostFreqSkinCount= 0,0
        else:
            b=np.array(emjText_skin)
            mostFreqSkin = b[np.argsort(b[:, 1])][-1][0]
            mostFreqSkinCount = int(b[np.argsort(b[:, 1])][-1][1])
 
        entry = {"date": datetime.datetime.utcnow(),\
            "created_at": tweet.created_at,\
        "text": tweet.text,\
        "retweet_count": tweet.retweet_count,\
     "favorite_count": tweet.favorite_count,\
     "lang": tweet.lang,\
     "geo": tweet.geo,"time_zone": tweet.user.time_zone,\
     "coordinates": tweet.coordinates, "name":tweet.user.name, "user_name":tweet.user.screen_name,\
     "emjText": emjText, "emjCount": emjCount, "emjTypes": emjTypes, "mostFreqEmoji": mostFreqEmoji,\
     "mostFreqEmojiCount": mostFreqEmojiCount, "prev_word": prev_word, "next_word": next_word,\
     "mostFreqWord": mostFreqWord, "mostFreqWordCount": mostFreqWordCount,\
     "surrounding_text": surrounding_text,\
     "emjText_skin": emjText_skin, "skinCount": skinCount, "skinTypes": skinTypes, "mostFreqSkin": mostFreqSkin,\
     "mostFreqSkinCount": mostFreqSkinCount, "newlineCount":newlineCount}

        collection.insert_one(entry).inserted_id
        print(tweet.text)
        
    write_emoji_usage(tweet,has_emoji)
    
    

In [576]:
#run the streamer
l=StdOutListener()
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)
myStream = tweepy.Stream(auth = api.auth, listener=l)
myStream.sample()

พี่แจบอมอาจจะจำไม่ได้ว่าเคยพูดอะไรไว้เพราะทุกอย่างที่พี่แจบอมทำ.....ใจมันสั่งมา💕 #พี่แจบอมไม่ชอบจับมือผู้ชาย https://t.co/tjn5FiiH5l
RT @yumu_34: 🌟お知らせ🌟

5月18日▶︎渋谷O-nest

アンダービースティー
2周年記念公演が決定しました✨
リアル2周年の日です！！

詳細は分かり次第お知らせします🙇🏻

この日はとっても大切な日。
今から予定を空けていただけたら
嬉しいです😭✨
よろし…
RT @ALDUBHashtagSQ: #ALDUB47thBOEAwards -2nd Spot PHL (NW), Rank 11 Worldwide Trends

👉 631K Tweets as of 11:45PM MLA TIME

@ALDub_RTeam ht…
RT @Korseries: มีการเปิดเผยว่า พัคโบกอม เป็นคนออกแบบโลโก้สินค้าOfficial งานแฟนมีตติ้งของซงจุงกิ😆💕 #SongJoongKiFanMeetingInSeoul https://t.c…
الله.. أُريد بش أن يدوم هذا التصالح مع نفسي,ورضاي,وأكتفائي ولا مُبالاتي. فاللهم أدمه💛.
RT @nkkivillafuerte: Natuto na🙈😂💙 #KATHNIELAt47thGMMSFAwards https://t.co/jdj235qY9m


KeyboardInterrupt: 

<h1> Calculate emoji usage </h1>

In [44]:
emoji_usage.find({"has_emoji":True}).count() / emoji_usage.find({"has_emoji":False}).count()

0.18618408845326842

<h1> test findig -1, +1, -5, +5 words around each emoji </h1>

In [520]:
original_text=u'💃🏽💃🏽 a 😍love my girl 🐶😍🐶so much 🐶😍🐶hello 🐶😍🐶🐶😍🐶💃🏽'
text=emoji_split(original_text)
print(text)

💃🏽 💃🏽 a 😍 love my girl 🐶 😍 🐶 so much 🐶 😍 🐶 hello 🐶 😍 🐶 🐶 😍 🐶 💃🏽


In [168]:
#build array of emoji strings
emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in ''.join([word if word in emj_codes else 'T' for word in text.split()]).rsplit('T') if emj_str != ''])
#analyze emoji strings:
d=collections.defaultdict(lambda:0)
for key in emj_str[:,0]:
    d[key]+=1
a=np.array([d.values(),d.keys(),[int(len(key)/2) for key in d.keys()]])
sort_index=a.argsort(axis=1)
emjstr=a[[1,sort_index[0]]][::-1]
emjstrCount=a[[0,sort_index[0]]][::-1]
emjstrLen=a[[2,sort_index[0]]][::-1]

array([u'1', u'3', u'3', u'8'], 
      dtype='<U16')

In [110]:
surrounding_text = [(emcode, text[:text.index(emcode)].split()[-1:],\
                    text[text.index(emcode)+2:].split()[:1],\
                    text[:text.index(emcode)].split()[-5:],\
                    text[text.index(emcode)+2:].split()[:5]) for emcode in emj_codes\
                      if (len(re.findall(emcode,text)) > 0)]

In [111]:
surrounding_text

[(u'\U0001f60d', [], [u'love'], [], [u'love', u'my', u'girl', u'so', u'much']),
 (u'\U0001f483',
  [u'\U0001f436'],
  [u'\U0001f3fd'],
  [u'\U0001f60d', u'\U0001f436', u'\U0001f436', u'\U0001f60d', u'\U0001f436'],
  [u'\U0001f3fd']),
 (u'\U0001f483\U0001f3fd',
  [u'\U0001f436'],
  [u'\U0001f3fd'],
  [u'\U0001f60d', u'\U0001f436', u'\U0001f436', u'\U0001f60d', u'\U0001f436'],
  [u'\U0001f3fd']),
 (u'\U0001f436',
  [u'much'],
  [u'\U0001f60d'],
  [u'love', u'my', u'girl', u'so', u'much'],
  [u'\U0001f60d', u'\U0001f436', u'hello', u'\U0001f436', u'\U0001f60d'])]

In [441]:
#add a space after the emoji, and remove double spaces
def emoji_split(text):
    for emcode in emj_codes:
        if (len(re.findall(emcode,text))) > 0:
            text=text.replace(emcode,emcode+' ')
    for skin in emj_codes_skin:
        text=text.replace(' '+skin,skin+' ') #put the skin codes back into place but add a space afterwards
    return ' '.join(text.split())

In [48]:
prev_word = ''.join(text[:text.index(u'🐶')].split()[-1:])
prev_word

u'much'

In [448]:
last_index=len(text)-(text[::-1].index(u'😍'[::-1]))-2
next_word = ''.join(text[last_index+2:].split()[:1])
next_word

u'love'

In [285]:
text=u'Baila como Juana la Cubana 💃🏽 YoYo'
text=emoji_split(text)

In [445]:
original_text=u'💃🏽💃🏽 a 😍love my girl 🐶😍🐶so much 🐶😍🐶hello 🐶😍🐶🐶😍🐶💃🏽'
text=emoji_split(original_text)
text

u'\U0001f483\U0001f3fd \U0001f483\U0001f3fd a \U0001f60d love my girl \U0001f436 \U0001f60d \U0001f436 so much \U0001f436 \U0001f60d \U0001f436 hello \U0001f436 \U0001f60d \U0001f436 \U0001f436 \U0001f60d \U0001f436 \U0001f483\U0001f3fd'

In [541]:
emjText=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes\
                      if (len(re.findall(emcode,text)) > 0)])
emjText

array([[u'\U0001f60d', u'5'],
       [u'\U0001f483', u'3'],
       [u'\U0001f483\U0001f3fd', u'3'],
       [u'\U0001f436', u'8']], 
      dtype='<U4')

In [542]:
#create arrays to save in SQL. Sorted by frequency
emojiLabel=emjText[np.argsort(emjText[:, 1])[::-1]][:,0] 
emojiCount=np.int32(emjText[np.argsort(emjText[:, 1])[::-1]][:,1])
emojiTypes=len(emojiCount)
emojiCountSum=sum(emojiCount)

In [448]:
def surroundingText(text,emojiLabel):
    return np.array([(emcode, ''.join(text[:text.index(emcode)].split()[-1:]),\
                    NextWord(text,emcode),\
                    ' '.join(text[:text.index(emcode)].split()[-7:]),\
                    ' '.join(text[text.index(emcode)+2:].split()[:7])) for emcode in emojiLabel\
                      if (len(re.findall(emcode,text)) > 0)])

In [449]:
surrounding_text=surroundingText(text,emojiLabel) #sorted by frequency
prev_word=surrounding_text[:,1]
next_word=surrounding_text[:,2]
prev_sentence=surrounding_text[:,3]
next_sentence=surrounding_text[:,4]

In [566]:
text='yoyo 🐶 man'

In [567]:
emj_str

array([], dtype=float64)

In [568]:
#build array of emoji strings
emj_str = np.array([(emj_str, int(len(emj_str)/2)) for emj_str in ''.join([word if word in emj_codes else 'T' for word in text.split()]).rsplit('T') if emj_str != ''])
#analyze emoji strings:
d=collections.defaultdict(lambda:0)
for key in emj_str[:,0]:
    d[key]+=1
a=np.array([d.values(),d.keys(),[int(len(key)/2) for key in d.keys()]])
#remove single emojis and double if skin code is included:
skin_cut=~np.bool8(((np.int32(a[2,:]))==2) & (np.array([sum([len(re.findall(emcode,val)) for emcode in emj_codes_skin]) for val in a[1,:]])))
multi_cut=(np.int32(a[2,:])>1) & skin_cut
a=a[:,multi_cut]
sort_index=a.argsort(axis=1)
emojistrLabel=a[[1,sort_index[0]]][::-1]
emojistrCount=np.int32(a[[0,sort_index[0]]][::-1])
emojistrLen=np.int32(a[[2,sort_index[0]]][::-1])
emojistrTypes=len(emojistrCount)
#add emjStr CountSum
surrounding_str_text=surroundingText(original_text,emojistrLabel) #sorted by frequency
str_prev_word=surrounding_str_text[:,1]
str_next_word=surrounding_str_text[:,2]
str_prev_sentence=surrounding_str_text[:,3]
str_next_sentence=surrounding_str_text[:,4]

/Users/attiladobi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


IndexError: too many indices for array

In [529]:
emojistrCount

array([2, 1, 1], dtype=int32)

In [572]:
original_text='yoyo 🐶 man'

In [573]:
#find emoji str patterns
pattern=np.array([(emcode, len(re.findall(emcode,original_text))) for emcode in emojistrLabel])
emojiPatternLabel=pattern[np.argsort(pattern[:, 1])[::-1]][:,0] 
emojiPatternCount=np.int32(pattern[np.argsort(pattern[:, 1])[::-1]][:,1])
emojiPatternLen=np.int32([np.int32(len(val)/2) for val in emojiPatternLabel])
emojiPatternTypes=len(emojiPatternCount)

In [574]:
pattern

array([[u'\U0001f436\U0001f60d\U0001f436', u'0'],
       [ u'\U0001f436\U0001f60d\U0001f436\U0001f436\U0001f60d\U0001f436\U0001f483\U0001f3fd',
        u'0'],
       [u'\U0001f483\U0001f3fd\U0001f483\U0001f3fd', u'0']], 
      dtype='<U16')

In [532]:
for val in emojiPatternLabel:
    print val

🐶😍🐶
💃🏽💃🏽
🐶😍🐶🐶😍🐶💃🏽


In [557]:
#now for emoji skin
text='yoyo 🐶😍🐶 man'

In [560]:
len(emjText_skin)

0

In [558]:
emjText_skin=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes_skin\
                      if (len(re.findall(emcode,text)) > 0)])
#create arrays to save in SQL. Sorted by frequency
emojiSkinLabel=emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,0] 
emojiSkinCount=np.int32(emjText_skin[np.argsort(emjText_skin[:, 1])[::-1]][:,1])
emojiSkinCountSum=sum(emojiSkinCount)
emojiSkinTypes=len(emojiSkinCount)

IndexError: too many indices for array

In [556]:
emjText_skin

array([[u'\U0001f3fd', u'3']], 
      dtype='<U2')

In [137]:
a=np.array(emjText)
mostFreqEmoji = a[np.argsort(a[:, 1])][-1][0]
mostFreqEmoji

u'\U0001f483\U0001f3fd'

In [138]:
prev_word = ''.join(text[:text.index(mostFreqEmoji)].split()[-1:])
prev_word

u'Cubana'

In [139]:
#Next Word
#if len(mostFreqEmoji)>2:
#    mostFreqEmoji=mostFreqEmoji[:2]
try:
    last_index=len(text)-(text[::-1].index(mostFreqEmoji[::-1]))-2
except ValueError:
    last_index=len(text)
next_word = ''.join(text[last_index+2:].split()[:1])
next_word

u'YoYo'

In [277]:
#text='hello'
emjText_skin=np.array([(emcode, len(re.findall(emcode,text))) for emcode in emj_codes_skin\
                      if (len(re.findall(emcode,text)) > 0)])
emjText_skin

[(u'\U0001f3fd', 1)]

In [489]:
skinCount=sum([item[1] for item in emjText_skin])

In [509]:
text.count ('\n')

3